In [0]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Path Of the file

In [0]:

!ls "/content/drive/My Drive/DS/Merna_Zakaria_Character_Recognition_DataSet/1/letters/a"
data = "/content/drive/My Drive/DS/Merna_Zakaria_Character_Recognition_DataSet/"

a0.csv	 a12.csv  a15.csv  a18.csv  a2.csv  a5.csv  a8.csv
a10.csv  a13.csv  a16.csv  a19.csv  a3.csv  a6.csv  a9.csv
a11.csv  a14.csv  a17.csv  a1.csv   a4.csv  a7.csv


In [0]:

import matplotlib.pyplot as plt
import numpy as np
import os
import keras
import pandas as pd
import csv
import tqdm
import string
import re
from google.colab import drive 
from tqdm import trange
alphabetics  = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']


number of digit labels--> 1819
number of letter  labels -->4734

Prepare dataset list & label list

In [0]:
def checker(row):
  if (len(row) > 0):
    return(row[0].isnumeric())
  return False


dataset = np.zeros((6553,1414))
labels = np.zeros((6553,1))

with open ('/content/drive/My Drive/DS/validFiles.csv') as csv_file:
  data = np.array([row for row in csv.reader(csv_file,delimiter=',')])
  for i in trange(len(data)):
    with open (data[i][0]) as csv_file_:
      csv_reader = csv.reader(csv_file_, delimiter=',')
      example = np.array([row[1:] for row in csv_reader if checker(row)],dtype =float).flatten()
      dataset[i] = np.pad(example, (0, 1414 - len(example)), 'constant')
      labels[i] = int(data[i][1])


100%|██████████| 6553/6553 [00:14<00:00, 447.12it/s]


split dataset to letters and digits

In [0]:
letters = dataset[1819:]
labels_letters = labels[1819:]
digits = dataset[:1819]
labels_digits=labels[:1819]


# split **function** 


In [0]:
def split(data, labels, split_percent):
  np.random.seed(42)
  print(data.shape)
  size = len(data)
  train_size = (size * split_percent)
  train_idx = np.random.choice(range(0, size), size=int(train_size), replace=False)
  test_idx = list(set(range(0,size))-set(train_idx))
  
  X_train = data[train_idx]
  X_test = data[test_idx]
  y_train = labels[train_idx]
  y_test = labels[test_idx]
  
  return X_train, y_train, X_test, y_test
#   return train_idx, test_idx

# Splitting

In [0]:

X_train, y_train, X_test, y_test = split(digits, labels_digits, 0.8)
print(np.array(X_train).shape)

#normalize the data to be in range(0,1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_X = scaler.fit_transform( X_train )
test_X = scaler.transform( X_test )


(1819, 1414)
(1455, 1414)


In [0]:
X_train, y_train, X_valid, y_valid = split(X_train, y_train, 0.9)

(3787, 1414)


In [0]:
# For some reason
y_train = y_train.flatten()
y_valid = y_valid.flatten()
y_test = y_test.flatten()

# Classification

# logistic regression

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Apply logistic regression model to training data
lr = LogisticRegression(penalty = 'l2', C = 0.1,random_state = 0,verbose =1)
lr.fit(X_train,y_train)

# Predict using model
v_pred = lr.predict(X_valid)
t_pred = lr.predict(X_train)


print("valid",accuracy_score(y_valid,v_pred))
print("train",accuracy_score(y_train,t_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[LibLinear]valid 0.1741424802110818
train 0.9501173708920188


In [0]:
t_pred = lr.predict(X_test)
print("train",accuracy_score(y_test,t_pred))

train 0.2774725274725275


# Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
clfRandomForest = RandomForestClassifier(n_estimators=800,min_samples_split = 4, min_samples_leaf = 2,max_features = 'sqrt',         
    max_depth = 200)
clfRandomForest.fit(X_train, y_train)  
output_labels = clfRandomForest.predict(X_valid)
    #calculate accuracy
train_labels = clfRandomForest.predict(X_train)
print( "train:",accuracy_score(y_train,train_labels))
print( "valid:",accuracy_score(y_valid,output_labels))

train: 1.0
valid: 0.5725593667546174


In [0]:
output_labels = clfRandomForest.predict(X_test)
print( "valid:",accuracy_score(y_test,output_labels))

valid: 0.5976768743400211


# KNN

In [0]:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import cross_val_score

#KNN Classifier
    neighbors = list(range(1,40))
    cv_scores = []
    #tune k paramter 
    for k in neighbors:
        knn = KNeighborsClassifier(n_neighbors=k)
        scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
        cv_scores.append(scores.mean())
    MSE = [1 - x for x in cv_scores]
    #optimal k that has min mse
    optimal_k = neighbors[MSE.index(min(MSE))]
    print(optimal_k)
    plt.plot(neighbors, cv_scores)
    plt.xlabel('Number of Neighbors K')
    plt.ylabel('Misclassification Error')
    plt.show()
    clf = KNeighborsClassifier(n_neighbors = optimal_k)
    clf.fit(X_train, y_train)
    output_labels = clf.predict(X_test)
    print(accuracy_score(output_labels,y_test))

KeyboardInterrupt: ignored

# Neural Networks

In [0]:
from keras.models import Sequential
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten,Dropout
from keras.optimizers import adam
model = Sequential()
model.add(Dense(10, input_dim=1414, activation='softmax'))

op = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=op, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 10)                14150     
Total params: 14,150
Trainable params: 14,150
Non-trainable params: 0
_________________________________________________________________


In [0]:
import keras
import keras.utils
from keras import utils as np_utils
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
model.fit(X_train, dummy_y, epochs=60, batch_size=50, verbose =2, validation_split=0.1, callbacks =[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train on 1309 samples, validate on 146 samples
Epoch 1/60
 - 1s - loss: 8.9153 - acc: 0.1345 - val_loss: 7.4152 - val_acc: 0.1507
Epoch 2/60
 - 0s - loss: 6.9066 - acc: 0.2017 - val_loss: 7.1232 - val_acc: 0.1781
Epoch 3/60
 - 0s - loss: 5.6093 - acc: 0.2765 - val_loss: 6.3839 - val_acc: 0.1918
Epoch 4/60
 - 0s - loss: 4.7518 - acc: 0.3331 - val_loss: 6.0224 - val_acc: 0.2055
Epoch 5/60
 - 0s - loss: 4.3525 - acc: 0.3193 - val_loss: 5.9547 - val_acc: 0.2260
Epoch 6/60
 - 0s - loss: 3.7544 - acc: 0.3858 - val_loss: 5.4191 - val_acc: 0.2466
Epoch 7/60
 - 0s - loss: 3.6744 - acc: 0.3873 - val_loss: 6.1601 - val_acc: 0.1918
Epoch 8/60
 - 0s - loss: 3.2105 - acc: 0.4263 - val_loss: 5.3844 - val_acc: 0.2055
Epoch 9/60
 - 0s - loss: 2.8791 - acc: 0.4614 - val_loss: 5.7871 - val_acc: 0.1986
Epoch 10/60
 - 0s - loss: 2.9721 - acc: 0.4423 - val_loss: 5.1776 - val_acc: 0.2192
Epoch 11/60
 - 0s - loss: 2.7044 - acc: 0.4675 - val_loss: 5.1788 - val_acc: 0.2192
Epoch 12/60
 - 0s - loss: 2.4756 - acc